In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
conn = psycopg2.connect(
    dbname= "Proyecto_ETL",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432"
)

In [3]:
cur = conn.cursor() # cursor me va a permitir ejecutar querys

In [4]:
cur.execute(query= "SELECT version();")
cur.fetchone()

('PostgreSQL 17rc1 on x86_64-windows, compiled by msvc-19.41.34120, 64-bit',)

TABLA CIUDAD

In [5]:
df_ciudad = pd.read_parquet("../data/ciudad.parquet")

In [6]:
tabla_ciudad = df_ciudad ["ciudad"].reset_index()
tabla_ciudad.head()

,index,ciudad
0,0,Madrid


In [7]:
# Creamos la consulta
insert_query = """
    INSERT INTO ciudad (nombre_ciudad)
    VALUES (%s)
"""

In [8]:
# Creamos una lista de tuplas con los valores a insertar
data_to_insert = []
for index, row in tabla_ciudad.iterrows():
    data_to_insert.append([row['ciudad'], 
                          ])


In [9]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLA EVENTOS

In [10]:
df_eventos = pd.read_parquet("../data/api_eventos.parquet")

In [11]:
print(df_eventos.columns)

Index(['title', 'url', 'postal-code', 'dirección', 'time', 'organization',
       'dtstart', 'dtend', 'id_ciudad'],
      dtype='object')


In [12]:
tabla_eventos = df_eventos [["title", "url", "postal-code", "dirección", "time", "dtstart", "dtend", "organization", "id_ciudad"]]
tabla_eventos.head()

,title,url,postal-code,dirección,time,dtstart,dtend,organization,id_ciudad
0,Black sad women,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,CALLE BOLTAÑA 27,19:00,2025-03-02,2025-03-02,Centro Cultural Buero Vallejo (San Blas - Cani...,1
1,Century Rock,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,CALLE BOLTAÑA 27,12:00,2025-03-02,2025-03-02,Centro Cultural Buero Vallejo (San Blas - Cani...,1
2,Circo Homónimo,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,CALLE ALIAGA 5,12:00,2025-03-02,2025-03-02,Centro Sociocultural Miguel de Cervantes (San ...,1
3,Clásicos del Cuento,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 14,10:30,2025-03-03,2025-03-03,Casa del Lector,1
4,Concurso de disfraces,http://www.madrid.es/sites/v/index.jsp?vgnextc...,0,No disponible,11:30,2025-03-02,2025-03-02,No disponible,1


In [13]:
# Creamos la consulta
insert_query = """
    INSERT INTO eventos ("nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "fecha_inicio", "fecha_fin", "organizacion", "id_ciudad")
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [14]:
# Creamos una lista de tuplas con los valores a insertar
data_to_insert = []
for index, row in tabla_eventos.iterrows():
    data_to_insert.append([row['title'], 
                           row['url'], 
                           row['postal-code'], 
                           row['dirección'], 
                           row['time'],
                           row['dtstart'],
                           row['dtend'],
                           row['organization'],
                           row['id_ciudad']
                           ])


In [15]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLA HOTELES

In [4]:
df_hoteles = pd.read_parquet("../data/hoteles.parquet")

In [17]:
tabla_hoteles = df_hoteles [["id_hotel", "nombre_hotel", "competencia", "estrellas", "id_ciudad"]]
tabla_hoteles.head()

,id_hotel,nombre_hotel,competencia,estrellas,id_ciudad
0,113,ibis Styles Madrid Prado,True,4.70,1
1,194,Novotel Madrid Center,True,4.60,1
2,1,Hotel Monte Verde,False,3.10,1
3,2,Hotel Brisas del Mar,False,3.09,1
4,3,Hotel Camino del Sol,False,3.07,1


In [18]:
# Creamos la consulta
insert_query = """
    INSERT INTO hoteles ("id_hotel", "nombre_hotel", "competencia", "valoracion", "id_ciudad")
    VALUES (%s, %s, %s, %s, %s)
"""

In [19]:
# Creamos una lista de tuplas con los valores a insertar
data_to_insert = []
for index, row in tabla_hoteles.iterrows():
    data_to_insert.append([row['id_hotel'], 
                           row['nombre_hotel'], 
                           row['competencia'], 
                           row['estrellas'], 
                           row['id_ciudad']
                           ])


In [20]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLA CLIENTES

In [21]:
df_clientes = pd.read_parquet("../data/clientes.parquet")

In [22]:
tabla_clientes = df_clientes [["id_cliente", "nombre", "apellido", "mail"]]
tabla_clientes.head()

,id_cliente,nombre,apellido,mail
0,1,Feliciana,Cantón,feliciana.cantón@example.com
1,2,Leonardo,Hierro,leonardo.hierro@example.com
2,3,Maite,Calatayud,maite.calatayud@example.com
3,4,Tecla,Bonet,tecla.bonet@example.com
4,5,Amílcar,Andrés,amílcar.andrés@example.com


In [23]:
# Creamos la consulta
insert_query = """
    INSERT INTO clientes ("id_cliente", "nombre", "apellido", "mail")
    VALUES (%s, %s, %s, %s)
"""

In [24]:
# Creamos una lista de tuplas con los valores a insertar
data_to_insert = []
for index, row in tabla_clientes.iterrows():
    data_to_insert.append([row['id_cliente'], 
                           row['nombre'], 
                           row['apellido'], 
                           row['mail'],
                           ])

In [25]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLA RESERVAS

In [37]:
df_reservas = pd.read_parquet("../data/dataset_hoteles_limpio.parquet")

In [38]:
tabla_reservas = df_reservas [["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel"]]
tabla_reservas.drop_duplicates(inplace=True)
tabla_reservas.duplicated().sum()

C:\Users\Javier\AppData\Local\Temp\ipykernel_16048\1660800889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_reservas.drop_duplicates(inplace=True)


np.int64(0)

In [39]:
# Creamos la consulta
insert_query = """
    INSERT INTO reservas ("id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel")
    VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [40]:
# Creamos una lista de tuplas con los valores a insertar
data_to_insert = []
for index, row in tabla_reservas.iterrows():
    data_to_insert.append([row['id_reserva'], 
                           row['fecha_reserva'] if pd.notna (row["fecha_reserva"]) else None,
                           row['inicio_estancia'] if pd.notna (row["inicio_estancia"]) else None,
                           row['final_estancia'] if pd.notna (row["final_estancia"]) else None,
                           row['precio_noche'],
                           row['id_cliente'],
                           row['id_hotel'],
                           ])

In [41]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [19]:
conn.rollback()

In [27]:
cur.close()
conn.close()